In [ ]:
import os
import sys
import json
import glob
import shutil
import deepdiff
import datetime
import rasterio
import numpy as np

sys.path.insert(0, '../')
from managers import managers

%load_ext autoreload
%autoreload 2

### Test DEM

In [ ]:
# original sierra map cmap
cmap_orig = [
    {'elevation': 0.0, 'color': (25, 125, 225)},
    {'elevation': 3.28, 'color': (110, 140, 100)},
    {'elevation': 328.0, 'color': (115, 150, 105)},
    {'elevation': 1312.0, 'color': (140, 155, 115)},
    {'elevation': 1640.0, 'color': (190, 204, 145)},
    {'elevation': 3280.0, 'color': (250, 250, 185)},
    {'elevation': 4920.0, 'color': (250, 205, 160)},
    {'elevation': 6560.0, 'color': (230, 180, 155)},
    {'elevation': 8200.0, 'color': (240, 200, 190)},
    {'elevation': 9840.0, 'color': (245, 220, 210)},
    {'elevation': 11480.0, 'color': (253, 235, 230)},
    {'elevation': 13120.0, 'color': (255, 255, 255)}]

In [ ]:
# DEM test
proj = managers.DEMProject(
    project_root='../projects/test-dem', 
    dataset_paths='../test/datasets/dem/n38w120/n38w120_100m.tif',
    reset=True)

proj.warp(proj.raw_datasets, res=200, crs='EPSG:3857')

In [ ]:
proj.hill_shade(proj.get_operation('last', 'warp'))
proj.slope_shade(proj.get_operation('last', 'warp'))
proj.texture_shade(proj.get_operation('last', 'warp'))
proj.color_relief(proj.get_operation('last', 'warp'), colormap=cmap_orig)

In [ ]:
proj.multiply_rgb(
    [proj.get_operation('last', 'hill_shade'), 
     proj.get_operation('last', 'color_relief')])
proj.save_props()

In [ ]:
proj._serialize()

In [ ]:
# DEM test
proj = managers.DEMProject(
    project_root='../projects/test-dem', 
    dataset_paths='../test/datasets/dem/n38w120/n38w120_100m.tif',
    reset=False)

In [ ]:
proj._serialize()

In [ ]:
proj.texture_shade(proj.get_operation('last', 'warp'))

### Test Landsat

In [ ]:
# test 'raw' landsat scenes
test_scenes = [
    '../test/datasets/landsat/LC08_L1TP_041034_20180916_20180928_01_T1',
    '../test/datasets/landsat/LC08_L1TP_042034_20180907_20180912_01_T1',]

In [ ]:
# real raw landsat scenes
full_scenes = [
    '/home/keith/landsat-data//LC08_L1TP_041034_20180916_20180928_01_T1',
    '/home/keith/landsat-data//LC08_L1TP_042034_20180907_20180912_01_T1',]

In [ ]:
# bounds for central Sierra 
latlon_bounds = [-119.8, 37.2, -117.8, 38.2]

In [ ]:
# check that we can reproduce these transformed bounds
landsat_crs_bounds = [251494.21, 4120732.28, 429952.78, 4228308.24]

tif = '../test/datasets/landsat/LC08_L1TP_041034_20180916_20180928_01_T1/LC08_L1TP_041034_20180916_20180928_01_T1_B1.TIF'
bounds = managers.utils.transform(latlon_bounds, tif)

bounds, landsat_crs_bounds

In [ ]:
# test landsatProject *with* resetting
proj = managers.LandsatProject(
    project_root='../projects/test-landsat',
    dataset_paths=test_scenes,
    reset=True)

proj.merge(proj.raw_datasets, bounds=latlon_bounds, res=400)

In [ ]:
proj.stack(proj.operations[0].destination, bands=[7, 5, 2])

In [ ]:
proj.autogain(proj.operations[-1].destination, percentile=99)

In [ ]:
proj.save_props()

In [ ]:
# load without resetting
proj = managers.LandsatProject(
    project_root='../projects/test-landsat',
    reset=False)

In [ ]:
proj.operations

In [ ]:
proj._serialize()

In [ ]:
# load real raw datasets
proj = managers.LandsatProject(
    project_root='/home/keith/landsat-data/projects/mono-lake/',
    dataset_paths=full_scenes,
    reset=False)

In [ ]:
# test operation class
s = managers.datasets.new_dataset('tif', 'test/path/one')
d = managers.datasets.new_dataset('tif', 'test/path/two')
op = managers.Operation(s, d, 'method_name', {'key': None}, 'commit_hash')

In [ ]:
managers.datasets.new_dataset('ned13', '../test/datasets/dem/n38w120/n38w120_100m.tif')

In [ ]:
managers.datasets.new_dataset('tif', '/media/keith/USGS_Backup/USGS/NED13/n39w123/', exists=True)